##DOWNLOADING DATASET FROM KAGGLE

In [ ]:
!mkdir -p /content/drive/MyDrive/hals/.kaggle
!cp /content/drive/MyDrive/hals/kaggle.json /content/drive/MyDrive/hals/.kaggle/

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/hals/.kaggle'


!kaggle datasets download -d aaronweymouth/nyc-rideshare-raw-data

100% 9.90G/9.91G [01:53<00:00, 83.1MB/s]
100% 9.91G/9.91G [01:53<00:00, 93.5MB/s]


##UNZIP

In [ ]:
!unzip /content/drive/MyDrive/hals/nyc-rideshare-raw-data.zip -d /content/drive/MyDrive/hals/nyc-rideshare-raw-data


Archive:  /content/drive/MyDrive/hals/nyc-rideshare-raw-data.zip
  inflating: /content/drive/MyDrive/hals/nyc-rideshare-raw-data/rideshare_data.parquet  
  inflating: /content/drive/MyDrive/hals/nyc-rideshare-raw-data/taxi_zone_lookup.csv  


## DOWNLOADING DATA FROM S3 OBJECT BUCKET

In [2]:
%%capture
!pip install boto3 graphframes


In [4]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'xxxxxxxxxxxxxxxxxxxxx' #replace with your AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' #replace with your AWS_SECRET_ACCESS_KEY

In [5]:
import boto3
import os

bucket_name = 'object-bucket-ec23421-09a0b3a1-a65e-481a-b18a-7debe6b9fb1f'

# Define your S3 configuration
s3_endpoint_url = 'https://ceph-a-object-rgw.comp-teach.qmul.ac.uk'
s3_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
s3_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']
bucket_name = 'data-repository-bkt'

# Create an S3 client
s3_client = boto3.client('s3',
                         endpoint_url=s3_endpoint_url,
                         aws_access_key_id=s3_access_key_id,
                         aws_secret_access_key=s3_secret_access_key)

# List objects in the ECS765 directory
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='ECS765/rideshare_2023/')

# Print the filenames in the ECS765 directory
print("Files in ECS765 directory:")
for obj in response.get('Contents', []):
    print(obj['Key'])


Files in ECS765 directory:
ECS765/rideshare_2023/aggregated_monthly_business_trips/_SUCCESS
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00000-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00031-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00054-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00069-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00119-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00125-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00138-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS765/rideshare_2023/aggregated_monthly_business_trips/part-00160-74becf6c-b0ca-42c8-86c9-284897e85991-c000.csv
ECS7

## Downloading files from s3 bucket

In [6]:
import boto3
import os

# Define your S3 configuration
s3_endpoint_url = 'https://ceph-a-object-rgw.comp-teach.qmul.ac.uk'
s3_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
s3_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']
bucket_name = 'data-repository-bkt'

# Create an S3 client
s3_client = boto3.client('s3',
                         endpoint_url=s3_endpoint_url,
                         aws_access_key_id=s3_access_key_id,
                         aws_secret_access_key=s3_secret_access_key)

# Specify the keys of the files to download
file_keys = [
    'ECS765/rideshare_2023/rideshare_data.csv',
    # 'ECS765/rideshare_2023/sample_data.csv',
    # 'ECS765/rideshare_2023/taxi_zone_lookup.csv'
]

# Download the specified files
for key in file_keys:
    file_name = key.split('/')[-1]  # Extract the file name
    local_file_path = f"/content/drive/MyDrive/hals/nyc-rideshare-raw-data/{file_name}"  # Specify the local file path to save
    s3_client.download_file(bucket_name, key, local_file_path)
    print(f"Downloaded file: {key}")


Downloaded file: ECS765/rideshare_2023/rideshare_data.csv


In [ ]:
!pip install pyspark

In [ ]:
import pandas as pd

# Define paths to the files
rideshare_data_path = '/content/drive/MyDrive/hals/nyc-rideshare-raw-data/rideshare_data.csv'
sample_data_path = '/content/drive/MyDrive/hals/nyc-rideshare-raw-data/sample_data.csv'
taxi_zone_lookup_path = '/content/drive/MyDrive/hals/nyc-rideshare-raw-data/taxi_zone_lookup.csv'

# Define chunk size (adjust based on memory availability)
chunksize = 10000  # Read 10000 rows at a time (you can adjust this value)

# Read the files in chunks
for filepath in [rideshare_data_path, sample_data_path, taxi_zone_lookup_path]:
    print(f"Reading file: {filepath}")
    for i, chunk in enumerate(pd.read_csv(filepath, chunksize=chunksize)):
        # Process the chunk of data here
        # For example, you can print the first few rows of each chunk
        print(f"Chunk {i+1} - Shape: {chunk.shape}")
        print(chunk.head())  # Show the first few rows of the chunk


## LOADING INTO SPARK

In [9]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=80d78ddc265cce8d3bf8d93cb466efcd12c74e1dd62d13ef23ea6b3f238a050f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime

# Create SparkSession
spark = SparkSession.builder \
    .appName("NYC Rideshare Analysis") \
    .getOrCreate()

# Load rideshare_data.csv
rides_df = spark.read.csv("/content/drive/MyDrive/hals/nyc-rideshare-raw-data/rideshare_data.csv", header=True)

# Load taxi_zone_lookup.csv
taxi_zone_df = spark.read.csv("/content/drive/MyDrive/hals/nyc-rideshare-raw-data/taxi_zone_lookup.csv", header=True)


In [18]:
from pyspark.sql.functions import col, from_unixtime

In [21]:
# Display schema of rides_df
print("Schema of rides_df:")
rides_df.printSchema()

# Display schema of taxi_zone_df
print("\nSchema of taxi_zone_df:")
taxi_zone_df.printSchema()


Schema of rides_df:
root
 |-- business: string (nullable = true)
 |-- pickup_location: string (nullable = true)
 |-- dropoff_location: string (nullable = true)
 |-- trip_length: string (nullable = true)
 |-- request_to_pickup: string (nullable = true)
 |-- total_ride_time: string (nullable = true)
 |-- on_scene_to_pickup: string (nullable = true)
 |-- on_scene_to_dropoff: string (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- passenger_fare: string (nullable = true)
 |-- driver_total_pay: string (nullable = true)
 |-- rideshare_profit: string (nullable = true)
 |-- hourly_rate: string (nullable = true)
 |-- dollars_per_mile: string (nullable = true)


Schema of taxi_zone_df:
root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



##Task 1 (MERGING)

In [60]:
from pyspark.sql.functions import col

# Step 1: Apply the 'join' function based on fields pickup_location and dropoff_location
# Alias the columns from taxi_zone_df appropriately to avoid duplicate column names
join_df = rides_df.alias("rides").join(taxi_zone_df.alias("pickup_zone"),
                                      col("rides.pickup_location") == col("pickup_zone.LocationID"), "left") \
                                 .join(taxi_zone_df.alias("dropoff_zone"),
                                      col("rides.dropoff_location") == col("dropoff_zone.LocationID"), "left") \
                                 .select("rides.*",
                                         col("pickup_zone.Borough").alias("Pickup_Borough"),
                                         col("pickup_zone.Zone").alias("Pickup_Zone"),
                                         col("pickup_zone.service_zone").alias("Pickup_service_zone"),
                                         col("dropoff_zone.Borough").alias("Dropoff_Borough"),
                                         col("dropoff_zone.Zone").alias("Dropoff_Zone"),
                                         col("dropoff_zone.service_zone").alias("Dropoff_service_zone"))

# Display the schema of the joined dataframe
join_df.printSchema()


root
 |-- business: string (nullable = true)
 |-- pickup_location: string (nullable = true)
 |-- dropoff_location: string (nullable = true)
 |-- trip_length: string (nullable = true)
 |-- request_to_pickup: string (nullable = true)
 |-- total_ride_time: string (nullable = true)
 |-- on_scene_to_pickup: string (nullable = true)
 |-- on_scene_to_dropoff: string (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- passenger_fare: string (nullable = true)
 |-- driver_total_pay: string (nullable = true)
 |-- rideshare_profit: string (nullable = true)
 |-- hourly_rate: string (nullable = true)
 |-- dollars_per_mile: string (nullable = true)
 |-- Pickup_Borough: string (nullable = true)
 |-- Pickup_Zone: string (nullable = true)
 |-- Pickup_service_zone: string (nullable = true)
 |-- Dropoff_Borough: string (nullable = true)
 |-- Dropoff_Zone: string (nullable = true)
 |-- Dropoff_service_zone: string (nullable = true)



In [61]:
print('Rides_df :', rides_df.columns)
print('taxi_zone_df :', taxi_zone_df.columns)

Rides_df : ['business', 'pickup_location', 'dropoff_location', 'trip_length', 'request_to_pickup', 'total_ride_time', 'on_scene_to_pickup', 'on_scene_to_dropoff', 'time_of_day', 'date', 'passenger_fare', 'driver_total_pay', 'rideshare_profit', 'hourly_rate', 'dollars_per_mile']
taxi_zone_df : ['LocationID', 'Borough', 'Zone', 'service_zone']


In [62]:
print('joined dataset:::',join_df.columns)


joined dataset::: ['business', 'pickup_location', 'dropoff_location', 'trip_length', 'request_to_pickup', 'total_ride_time', 'on_scene_to_pickup', 'on_scene_to_dropoff', 'time_of_day', 'date', 'passenger_fare', 'driver_total_pay', 'rideshare_profit', 'hourly_rate', 'dollars_per_mile', 'Pickup_Borough', 'Pickup_Zone', 'Pickup_service_zone', 'Dropoff_Borough', 'Dropoff_Zone', 'Dropoff_service_zone']


In [63]:
print('Rides_df :')
rides_df.show(5)
print('taxi_zone_df :')
taxi_zone_df.show(5)

Rides_df :
+--------+---------------+----------------+-----------+-----------------+---------------+------------------+-------------------+-----------+----------+--------------+----------------+----------------+-----------+----------------+
|business|pickup_location|dropoff_location|trip_length|request_to_pickup|total_ride_time|on_scene_to_pickup|on_scene_to_dropoff|time_of_day|      date|passenger_fare|driver_total_pay|rideshare_profit|hourly_rate|dollars_per_mile|
+--------+---------------+----------------+-----------+-----------------+---------------+------------------+-------------------+-----------+----------+--------------+----------------+----------------+-----------+----------------+
|    Uber|            151|             244|       4.98|            226.0|          761.0|              19.0|              780.0|    morning|1684713600|         22.82|           13.69|            9.13|      63.18|            2.75|
|    Uber|            244|              78|       4.35|            19

In [64]:
# Show the first few rows of the joined dataframe
join_df.show(5)

+--------+---------------+----------------+-----------+-----------------+---------------+------------------+-------------------+-----------+----------+--------------+----------------+----------------+-----------+----------------+--------------+--------------------+-------------------+---------------+--------------------+--------------------+
|business|pickup_location|dropoff_location|trip_length|request_to_pickup|total_ride_time|on_scene_to_pickup|on_scene_to_dropoff|time_of_day|      date|passenger_fare|driver_total_pay|rideshare_profit|hourly_rate|dollars_per_mile|Pickup_Borough|         Pickup_Zone|Pickup_service_zone|Dropoff_Borough|        Dropoff_Zone|Dropoff_service_zone|
+--------+---------------+----------------+-----------+-----------------+---------------+------------------+-------------------+-----------+----------+--------------+----------------+----------------+-----------+----------------+--------------+--------------------+-------------------+---------------+-----------

####Convert UNIX timestamp to "yyyy-MM-dd" format

In [65]:
from pyspark.sql.functions import from_unixtime

# Convert UNIX timestamp to date format
join_df = join_df.withColumn("date", from_unixtime("date", "yyyy-MM-dd"))

####Print the number of rows and schema of the new DataFrame

In [66]:
# Count the number of rows
row_count = join_df.count()
print("Number of Rows:", row_count)

# Print the schema of the DataFrame
join_df.printSchema()


Number of Rows: 69725864
root
 |-- business: string (nullable = true)
 |-- pickup_location: string (nullable = true)
 |-- dropoff_location: string (nullable = true)
 |-- trip_length: string (nullable = true)
 |-- request_to_pickup: string (nullable = true)
 |-- total_ride_time: string (nullable = true)
 |-- on_scene_to_pickup: string (nullable = true)
 |-- on_scene_to_dropoff: string (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- passenger_fare: string (nullable = true)
 |-- driver_total_pay: string (nullable = true)
 |-- rideshare_profit: string (nullable = true)
 |-- hourly_rate: string (nullable = true)
 |-- dollars_per_mile: string (nullable = true)
 |-- Pickup_Borough: string (nullable = true)
 |-- Pickup_Zone: string (nullable = true)
 |-- Pickup_service_zone: string (nullable = true)
 |-- Dropoff_Borough: string (nullable = true)
 |-- Dropoff_Zone: string (nullable = true)
 |-- Dropoff_service_zone: string (nullable = true)
